# TensorFlow RNN Net for Classification

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Load dataset

In [2]:
mnist = input_data.read_data_sets('./MNIST_data', one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


---

# Build Network

Hyperparameters

In [16]:
lr = 0.0001
training_iters = 100000
batch_size = 128

Network Params

In [4]:
## MNIST dataset 28*28
n_inputs = 28 
n_steps = 28
n_classes = 10
n_hidden_units = 128

Placeholders

In [5]:
x = tf.placeholder(shape=[None, 784], dtype=tf.float32, name='input')
y = tf.placeholder(shape=[None, 10],  dtype=tf.float32, name='digit')

Weights and biases

In [6]:
Weights = {
    'in' : tf.Variable(tf.random_normal(shape=[n_inputs, n_hidden_units],  dtype=tf.float32, name='in_Weight')),
    'out': tf.Variable(tf.random_normal(shape=[n_hidden_units, n_classes], dtype=tf.float32, name='out_Weight'))
}

biases = {
    'in' : tf.Variable(tf.constant(0.1, shape=[n_hidden_units]), name='in_bais'),
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes])     , name='out_bais')
}

Input layer before rnn

In [7]:
# X(num_batches, 28 steps, 28 inputs)
# --> X(num_batches*28, 28 inputs)
x_reshape = tf.reshape(x, [-1, n_inputs])
# --> X_in(num_batches*128, 28 inputs)
x_in = tf.matmul(x_reshape, Weights['in']) + biases['in']
# --> X_in(num_batches, 28 steps, 128 inputs)
x_in = tf.reshape(x_in, [-1, n_steps, n_hidden_units])

RNN

In [8]:
lstm_cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=n_hidden_units, 
    forget_bias=1.0, 
    state_is_tuple=True
)
# state_is_tuple = True
# LSTM state is divided into two parts (c_state, m_state)
initial_state = lstm_cell.zero_state(batch_size, dtype=tf.float32)
# Compute rnn
outputs, states = tf.nn.dynamic_rnn(
    cell=lstm_cell, 
    inputs=x_in, 
    initial_state=initial_state
)

rnn_output = states[1]

# or
# transpose : (batch_size, steps, n_hidden_units) -> (batch_size, n_hidden_units, steps)
# unpack    : outputs as list [(batch outputs)...] * steps

# outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
# rnn_output = outputs[-1]

In [9]:
prediction = tf.matmul(rnn_output, Weights['out']) + biases['out']

train and loss

In [10]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=prediction)
train = tf.train.AdamOptimizer(lr).minimize(loss)

Accuracy

In [14]:
accuracy = tf.metrics.accuracy(labels=tf.argmax(y, axis=1), predictions=tf.argmax(prediction, axis=1))[1]

---

# Train the network

In [17]:
with tf.Session() as sess:
    
    # Init step
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    # Train
    step=0
    while step*batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train, feed_dict={
            x: batch_xs,
            y: batch_ys
        })
        if step%10==0:
            batch_xs, batch_ys = mnist.validation.next_batch(batch_size)
            print(sess.run(accuracy, feed_dict={
                x: batch_xs,
                y: batch_ys
            }))
        step += 1

0.101562
0.171875
0.239583
0.28125
0.323438
0.363281
0.405134
0.435547
0.459201
0.478906
0.50142
0.51888
0.536659
0.545759
0.558333
0.569824
0.58318
0.59592
0.60773
0.616797
0.622396
0.630327
0.637568
0.644857
0.650937
0.655649
0.662616
0.667969
0.672953
0.677083
0.681956
0.6875
0.691761
0.695312
0.699554
0.703342
0.707348
0.710938
0.715545
0.718359
0.72218
0.724702
0.727471
0.731534
0.736111
0.738961
0.742354
0.745931
0.748565
0.750156
0.752451
0.754958
0.757518
0.759983
0.763636
0.765765
0.768092
0.770474
0.772643
0.775391
0.777152
0.778226
0.780506
0.782471
0.784495
0.786222
0.78743
0.788488
0.789742
0.791295
0.792804
0.794705
0.796875
0.797614
0.799375
0.800473
0.801542
0.802885
0.804391
